# Teste da API Comtrade

Info: https://comtradedeveloper.un.org

Specs: https://comtradedeveloper.un.org/api-details#api=comtrade-v1&operation=get-get



In [51]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [52]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [53]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Aqui colocar os parâmetros gerais que não mudam.

In [54]:
baseUrl = "https://comtradeapi.un.org/public/v1"

m49_codes_cn_plp ={
    "024":"Angola",
    "076":"Brazil",
    "132":"Cabo Verde",
    "156":"China",
    "344":"China, Hong Kong",
    "446":"China, Macau",
    "226":"Guiné Equatorial",
    "624":"Guiné-Bissau",
    "508":"Moçambique",
    "620":"Portugal",
    "678":"São Tomé e Príncipe",
    "626":"Timor-Leste"
}
m49_china = "156"
m49_hong_kong = "344"
m49_macau = "446"
m49_plp = set(m49_codes_cn_plp.keys())-{m49_china,m49_hong_kong,m49_macau}
m49_plp_list = ",".join(m49_plp)

Aqui os parâmetros de cada chamada à API.

In [55]:
typeCode = "C" # C for commodities, S for Services
freqCode = "A" # A for annual and M for monthly
clCode = "HS" #Trade classifications Available values : HS, SS, B4, B5, EB, EB10, EB02, EBSDMX
reporterCode = m49_china # China, HK 344, Macau 446, see https://unstats.un.org/unsd/methodology/m49/
partnerCode = m49_plp_list
period="2020"
cmdCode="AG2" # TOTAL, AG2 (HS level 2)
flowCode = "M" # M=import, X=export,outros: RX, RM, MIP, XIP, MOP, XOP, MIF, XIF, DX, FM


In [59]:
import requests
requestUrl=f"{baseUrl}/preview/{typeCode}/{freqCode}/{clCode}"
resp = requests.get(requestUrl,
        {
        'reporterCode':m49_china,
        'period':period,
        'partnerCode':m49_plp_list,
        'cmdCode':cmdCode,
        'flowCode':flowCode
        },
        timeout=10)

In [60]:
import json
import pandas as pd

results = json.loads(resp.content)['data']
dados = pd.DataFrame(results)
# convert result country codes to three-digit codes
dados.reporterCode = dados.reporterCode.astype('str').str.zfill(3)
dados.partnerCode = dados.partnerCode.astype('str').str.zfill(3)
# print(dados.columns.values)

In [61]:


dados.reporterDesc = dados.reporterCode.map(m49_codes_cn_plp)
dados.partnerDesc = dados.partnerCode.map(m49_codes_cn_plp)
dados['primaryValueFormated'] = dados.primaryValue.map('{:,}'.format)
cols = ['typeCode','cmdCode','freqCode','refYear','reporterDesc','partnerDesc','flowCode','primaryValueFormated','primaryValue']

dados[cols]

,typeCode,cmdCode,freqCode,refYear,reporterDesc,partnerDesc,flowCode,primaryValueFormated,primaryValue
0,C,21,A,2020,China,Angola,M,155.0,1.550000e+02
1,C,22,A,2020,China,Angola,M,"94,554.0",9.455400e+04
2,C,25,A,2020,China,Angola,M,"29,428,961.0",2.942896e+07
3,C,26,A,2020,China,Angola,M,"6,763,117.0",6.763117e+06
4,C,27,A,2020,China,Angola,M,"14,686,950,196.0",1.468695e+10
...,...,...,...,...,...,...,...,...,...
256,C,84,A,2020,China,São Tomé e Príncipe,M,"37,730.0",3.773000e+04
257,C,85,A,2020,China,São Tomé e Príncipe,M,989.0,9.890000e+02
258,C,87,A,2020,China,São Tomé e Príncipe,M,226.0,2.260000e+02
259,C,90,A,2020,China,São Tomé e Príncipe,M,"5,518.0",5.518000e+03


In [36]:
dados[cols].to_excel(f"./dados_comtrade_{period}_{cmdCode}.xlsx")